<a href="https://colab.research.google.com/github/choki0715/lecture/blob/master/bitandbytes_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers bitsandbytes


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# GPU 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type != "cuda":
    raise ValueError("CUDA 장치를 사용 중이어야 합니다. 현재 장치: CPU")

# 모델 이름
model_name = 'gpt2'



In [3]:
# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 파라미터 확인
for name, param in model.named_parameters():
    print(f"Parameter: {name}, dtype: {param.dtype}, shape: {param.shape}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Parameter: transformer.wte.weight, dtype: torch.float32, shape: torch.Size([50257, 768])
Parameter: transformer.wpe.weight, dtype: torch.float32, shape: torch.Size([1024, 768])
Parameter: transformer.h.0.ln_1.weight, dtype: torch.float32, shape: torch.Size([768])
Parameter: transformer.h.0.ln_1.bias, dtype: torch.float32, shape: torch.Size([768])
Parameter: transformer.h.0.attn.c_attn.weight, dtype: torch.float32, shape: torch.Size([768, 2304])
Parameter: transformer.h.0.attn.c_attn.bias, dtype: torch.float32, shape: torch.Size([2304])
Parameter: transformer.h.0.attn.c_proj.weight, dtype: torch.float32, shape: torch.Size([768, 768])
Parameter: transformer.h.0.attn.c_proj.bias, dtype: torch.float32, shape: torch.Size([768])
Parameter: transformer.h.0.ln_2.weight, dtype: torch.float32, shape: torch.Size([768])
Parameter: transformer.h.0.ln_2.bias, dtype: torch.float32, shape: torch.Size([768])
Parameter: transformer.h.0.mlp.c_fc.weight, dtype: torch.float32, shape: torch.Size([768, 3072]

In [5]:
# 예제 입력

model.to(device)

input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# 양자화된 모델로 텍스트 생성 수행
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=50, num_return_sequences=1)

# 출력 디코딩
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Generated text: {generated_text}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text: Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a


In [7]:

# bitandbytes 설정
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8-bit 양자화 사용
    llm_int8_threshold=6.0  # 양자화 임계값 설정
)

# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Remove the line below as the model is already on the GPU due to BitsAndBytes
# model.to(device)

# 예제 입력
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# 양자화된 모델로 텍스트 생성 수행
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=50, num_return_sequences=1)

# 출력 디코딩
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Generated text: {generated_text}")



`low_cpu_mem_usage` was None, now set to True since model is quantized.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text: Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger.

The world was a place of great danger, and the world


In [8]:

# 모델 파라미터 확인
for name, param in model.named_parameters():
    print(f"Parameter: {name}, dtype: {param.dtype}, shape: {param.shape}")

Parameter: transformer.wte.weight, dtype: torch.float16, shape: torch.Size([50257, 768])
Parameter: transformer.wpe.weight, dtype: torch.float16, shape: torch.Size([1024, 768])
Parameter: transformer.h.0.ln_1.weight, dtype: torch.float16, shape: torch.Size([768])
Parameter: transformer.h.0.ln_1.bias, dtype: torch.float16, shape: torch.Size([768])
Parameter: transformer.h.0.attn.c_attn.weight, dtype: torch.int8, shape: torch.Size([2304, 768])
Parameter: transformer.h.0.attn.c_attn.bias, dtype: torch.float16, shape: torch.Size([2304])
Parameter: transformer.h.0.attn.c_proj.weight, dtype: torch.int8, shape: torch.Size([768, 768])
Parameter: transformer.h.0.attn.c_proj.bias, dtype: torch.float16, shape: torch.Size([768])
Parameter: transformer.h.0.ln_2.weight, dtype: torch.float16, shape: torch.Size([768])
Parameter: transformer.h.0.ln_2.bias, dtype: torch.float16, shape: torch.Size([768])
Parameter: transformer.h.0.mlp.c_fc.weight, dtype: torch.int8, shape: torch.Size([3072, 768])
Paramet